Importing data

In [ ]:
 from google.colab import drive
 drive.mount('/content/gdrive')

MessageError: ignored

Importing the libraries

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import brown
import gensim
import gensim.downloader as api
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split


Downloading the dataset

In [ ]:
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('punkt')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('punkt')

brown_tagged_sents = brown.tagged_sents(tagset='universal')

lowered_tagged_sents = [[(word.lower(), tag) for (word, tag) in sent] for sent in brown_tagged_sents]

tagset=['.', 'ADJ','ADP', 'ADV', 'CONJ','DET','NOUN','NUM','PRON','PRT','VERB','X']

tag_to_ix = {tag: i for i, tag in enumerate(tagset)}
# Set the dimensionality of the word vectors
dim = 300

# Load the pre-trained GloVe model with dimensionality set to `dim`
model_wordvec = api.load('glove-wiki-gigaword-' + str(dim))



# Get the Brown Corpus sentences with Universal Tagset tags
brown_tagged_sents = brown.tagged_sents(tagset='universal')

# Lowercase the words in the tagged sentences
lowered_tagged_sents = [[(word.lower(), tag) for (word, tag) in sent] for sent in brown_tagged_sents]

# Convert each word to its word vector
unique_tags=set()
word_vectors = []
for sent in lowered_tagged_sents:
    for word, tag in sent:
        # Check if the word is in the GloVe vocabulary
        if word in model_wordvec.vocab:
            # Add the word vector to the sentence vector
            word2vec=model_wordvec[word]
            word_vectors.append((word2vec,tag_to_ix[tag]))
            unique_tags.add(tag)

        else:
            # If the word is not in the vocabulary, add a random vector with mean 0 and standard deviation 1
            word2vec=np.random.normal(0, 1, dim)
            word_vectors.append((word2vec,tag_to_ix[tag]))
            unique_tags.add(tag)

class POS_NN(nn.Module):

   def __init__(self, input_size, hidden_size, output_size):
        super(POS_NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

   def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set the input dimension to the size of the word vectors
input_dim = dim

# Set the output dimension to the number of tags in the Universal Tagset
output_dim = len(tagset)

# Set the dimension of the hidden layer
hidden_dim = 128

# Define the model
model = POS_NN(input_dim, hidden_dim, output_dim)


train_size = int(0.8 * len(word_vectors))
train_sents = word_vectors[:train_size]
test_sents = word_vectors[train_size:]

train_loader = torch.utils.data.DataLoader(train_sents, batch_size=32, shuffle=True)

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 200
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        # Unpack the batch into separate lists of words and tags
        words = batch[0]
        tags = batch[1]

        # Convert the sentences and tags to PyTorch tensors
        word_tensor = torch.tensor(words, dtype=torch.long).float()
        tag_tensor = torch.tensor(tags, dtype=torch.long)

        # Clear the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(word_tensor)

        # Calculate the loss
        loss = loss_function(outputs, tag_tensor)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add the loss to the total loss for this epoch
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    avg_loss = total_loss / len(train_loader)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))

torch.save(model.state_dict())

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: ignored

In [ ]:

brown_tagged_sents = brown.tagged_sents(tagset='universal')

print("number of sentences are: ",len(brown_tagged_sents) )
print("example of sentence and POS tagging: ",brown_tagged_sents[0])


LookupError: ignored

Pre-processing

In [ ]:
# Tokenize and lowercase the sentences
lowered_tagged_sents = [[(word.lower(), tag) for (word, tag) in sent] for sent in brown_tagged_sents]
print(len(lowered_tagged_sents))
print(lowered_tagged_sents[0])


57340
[('the', 'DET'), ('fulton', 'NOUN'), ('county', 'NOUN'), ('grand', 'ADJ'), ('jury', 'NOUN'), ('said', 'VERB'), ('friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]


Word Vectors

In [ ]:
tagset=['.', 'ADJ','ADP', 'ADV', 'CONJ','DET','NOUN','NUM','PRON','PRT','VERB','X']

In [ ]:
tag_to_ix = {tag: i for i, tag in enumerate(tagset)}

In [ ]:
# Set the dimensionality of the word vectors
dim = 300

# Load the pre-trained GloVe model with dimensionality set to `dim`
model_wordvec = api.load('glove-wiki-gigaword-' + str(dim))

In [ ]:


# Get the Brown Corpus sentences with Universal Tagset tags
brown_tagged_sents = brown.tagged_sents(tagset='universal')

# Lowercase the words in the tagged sentences
lowered_tagged_sents = [[(word.lower(), tag) for (word, tag) in sent] for sent in brown_tagged_sents]

# Convert each word to its word vector
unique_tags=set()
word_vectors = []
for sent in lowered_tagged_sents:
    for word, tag in sent:
        # Check if the word is in the GloVe vocabulary
        if word in model_wordvec.vocab:
            # Add the word vector to the sentence vector
            word2vec=model_wordvec[word]
            word_vectors.append((word2vec,tag_to_ix[tag]))
            unique_tags.add(tag)

        else:
            # If the word is not in the vocabulary, add a random vector with mean 0 and standard deviation 1
            word2vec=np.random.normal(0, 1, dim)
            word_vectors.append((word2vec,tag_to_ix[tag]))
            unique_tags.add(tag)


In [ ]:
word_vectors[0]

Nueral Model Architecture and Parameters

In [ ]:
class POS_NN(nn.Module):

   def __init__(self, input_size, hidden_size, output_size):
        super(POS_NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

   def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



Initiating Model

In [ ]:
# Set the input dimension to the size of the word vectors
input_dim = dim

# Set the output dimension to the number of tags in the Universal Tagset
output_dim = len(tagset)

# Set the dimension of the hidden layer
hidden_dim = 128

# Define the model
model = POS_NN(input_dim, hidden_dim, output_dim)


Preparing the data

In [ ]:
train_size = int(0.8 * len(word_vectors))
train_sents = word_vectors[:train_size]
test_sents = word_vectors[train_size:]

Training

In [ ]:
train_loader = torch.utils.data.DataLoader(train_sents, batch_size=32, shuffle=True)

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 200
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        # Unpack the batch into separate lists of words and tags
        words = batch[0]
        tags = batch[1]

        # Convert the sentences and tags to PyTorch tensors
        word_tensor = torch.tensor(words, dtype=torch.long).float()
        tag_tensor = torch.tensor(tags, dtype=torch.long)

        # Clear the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(word_tensor)

        # Calculate the loss
        loss = loss_function(outputs, tag_tensor)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add the loss to the total loss for this epoch
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    avg_loss = total_loss / len(train_loader)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))

torch.save(model.state_dict())

Testing

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for sentence, tags in test_sents:
    # Convert the sentence to a PyTorch tensor
    sentence_tensor = torch.tensor(sentence, dtype=torch.float32)
        # Convert the tag to a PyTorch tensor
    tag_tensor = torch.tensor(tags, dtype=torch.long)

    # Forward pass
    outputs = model(sentence_tensor)

    # Get the predicted tag
    _, predicted = torch.max(outputs.data, 0)

    # Update the count of correct predictions and the total count
    total += 1
    if predicted == tag_tensor:
        correct += 1
    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))


Giving context



In [ ]:
# Set the dimensionality of the word vectors
dim = 300
model_wordvec = api.load('glove-wiki-gigaword-' + str(dim))


[===============================================---] 95.6% 359.6/376.1MB downloaded


In [ ]:


nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('punkt')



# Load the pre-trained GloVe model

# Get the Brown Corpus sentences with Universal Tagset tags
brown_tagged_sents = brown.tagged_sents(tagset='universal')

# Lowercase the words in the tagged sentences
lowered_tagged_sents = [[(word.lower(), tag) for (word, tag) in sent] for sent in brown_tagged_sents]

# Set the tagset
tagset = ['.', 'ADJ','ADP', 'ADV', 'CONJ','DET','NOUN','NUM','PRON','PRT','VERB','X']
tag_to_ix = {tag: i for i, tag in enumerate(tagset)}



# Define a function to get the context of a word in a sentence
def get_context(sent, index, context_size):
    start = max(0, index - context_size)
    end = index
    context = sent[start:end]
    return context

# Convert each word to its word vector along with context
word_vectors = []
unique_tags = set()

# Set the context size
context_size = 2

for sent in lowered_tagged_sents:
    for i, (word, tag) in enumerate(sent):
        # Get the context of the word in the sentence
        context = get_context(sent, i, context_size)
        context_vecs = []
        for context_word, _ in context:
            # Check if the context word is in the GloVe vocabulary
            if context_word in model_wordvec.vocab:
                context_vecs.append(model_wordvec[context_word])
            else:
                # If the context word is not in the vocabulary, add a vector of zeros
                context_vecs.append(np.zeros(dim))
        if len(context_vecs)!= context_size:
          extra=(context_size)-len(context_vecs)
          context_vecs.insert(0,np.zeros(dim*extra))
        # Check if the word is in the GloVe vocabulary
        if word in model_wordvec.vocab:
            word2vec = model_wordvec[word]
            word_vectors.append((np.concatenate(context_vecs + [word2vec]), tag_to_ix[tag]))
            unique_tags.add(tag)
        else:
            # If the word is not in the vocabulary, add a random vector with mean 0 and standard deviation 1
            word2vec = np.random.normal(0, 1, dim)
            word_vectors.append((np.concatenate(context_vecs + [word2vec]), tag_to_ix[tag]))
            unique_tags.add(tag)

class POS_NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(POS_NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set the input dimension to the size of


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Set the input dimension to the size of the concatenated word vectors and context vectors
input_dim = dim * (context_size + 1)

# Set the output dimension to the number of tags in the Universal Tagset
output_dim = len(tagset)

# Set the dimension of the hidden layer
hidden_dim = 128

# Define the model
model = POS_NN(input_dim, hidden_dim, output_dim)



In [ ]:
train_size = int(0.8 * len(word_vectors))
train_sents = word_vectors[:train_size]
test_sents = word_vectors[train_size:]

train_loader = torch.utils.data.DataLoader(train_sents, batch_size=32, shuffle=True)

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 50
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        # Unpack the batch into separate lists of words and tags
        words = batch[0]
        tags = batch[1]

        # Convert the sentences and tags to PyTorch tensors
        word_tensor = torch.tensor(words, dtype=torch.long).float()
        tag_tensor = torch.tensor(tags, dtype=torch.long)

        # Clear the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(word_tensor)

        # Calculate the loss
        loss = loss_function(outputs, tag_tensor)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add the loss to the total loss for this epoch
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    avg_loss = total_loss / len(train_loader)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))

torch.save(model.state_dict())

<ipython-input-9-ad101de1610d>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  word_tensor = torch.tensor(words, dtype=torch.long).float()
<ipython-input-9-ad101de1610d>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tag_tensor = torch.tensor(tags, dtype=torch.long)


Epoch [1/50], Loss: 1.2389
Epoch [2/50], Loss: 1.1875
Epoch [3/50], Loss: 1.1707
Epoch [4/50], Loss: 1.1632
Epoch [5/50], Loss: 1.1530
Epoch [6/50], Loss: 1.1477
Epoch [7/50], Loss: 1.1427
Epoch [8/50], Loss: 1.1389
Epoch [9/50], Loss: 1.1306
Epoch [10/50], Loss: 1.1277
Epoch [11/50], Loss: 1.1230
Epoch [12/50], Loss: 1.1204
Epoch [13/50], Loss: 1.1188
Epoch [14/50], Loss: 1.1161
Epoch [15/50], Loss: 1.1141


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, fbeta_score

def evaluate(model, test_sentences):

    y_true = []
    y_pred = []

    for sentence in test_sentences:

        word  = sentence[0]
        target=sentence[1]
        word_tensor = torch.tensor(word, dtype=torch.long).float()
        tag_tensor = torch.tensor(target, dtype=torch.long)
        # print(word_tensor.shape)
        # print(tag_tensor.shape)

        with torch.no_grad():
            output = model(word_tensor)

            _, predicted = torch.max(output.data, 0)
            # print(target,predicted)
            y_true.append(target)
            y_pred.append(predicted.item())

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    f0_5 = fbeta_score(y_true, y_pred, beta=0.5, average='weighted')
    f2 = fbeta_score(y_true, y_pred, beta=2, average='weighted')
    accuracy = sum([y_true[i] == y_pred[i] for i in range(len(y_true))]) / len(y_true)


    return precision, recall, f1, f0_5, f2, accuracy

In [ ]:
precision, recall, f1, f0_5, f2, accuracy=evaluate(model,test_sents)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, fbeta_score

def evaluate_class(model, test_sentences):

    y_true = []
    y_pred = []

    for sentence in test_sentences:

        word  = sentence[0]
        target=sentence[1]
        word_tensor = torch.tensor(word, dtype=torch.long).float()
        tag_tensor = torch.tensor(target, dtype=torch.long)
        # print(word_tensor.shape)
        # print(tag_tensor.shape)

        with torch.no_grad():
            output = model(word_tensor)

            _, predicted = torch.max(output.data, 0)
            # print(target,predicted)
            y_true.append(target)
            y_pred.append(predicted.item())

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=1)
    print(y_pred,y_true)

    return precision, recall, f1

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, fbeta_score

def cm(model, test_sentences):

    y_true = []
    y_pred = []

    for sentence in test_sentences:

        word  = sentence[0]
        target=sentence[1]
        word_tensor = torch.tensor(word, dtype=torch.long).float()
        tag_tensor = torch.tensor(target, dtype=torch.long)
        # print(word_tensor.shape)
        # print(tag_tensor.shape)

        with torch.no_grad():
            output = model(word_tensor)

            _, predicted = torch.max(output.data, 0)
            # print(target,predicted)
            y_true.append(target)
            y_pred.append(predicted.item())


    labels=['.', 'ADJ','ADP', 'ADV', 'CONJ','DET','NOUN','NUM','PRON','PRT','VERB','X']
    cm = confusion_matrix(y_true, y_pred)

    # Normalize confusion matrix
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Plot confusion matrix heatmap
    sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=labels, yticklabels=labels)

    # Add labels and title
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
def model_statstics(precision, recall, f1, f0_5, f2, accuracy,precision_class,recall_class,f1_class):
  print("Global statistics")
  print("\n")
  print("Accuracy: ",accuracy)
  print("Precision: ",precision)
  print("Recall: ",recall)
  print("F Score")
  print("f1 score: ",f1)
  print("f0_5 score: ",f0_5)
  print("f2_score: ",f2)
  print("\n")
  print("Class wise stats")
  print("\n")

  for i in range(len(precision_class)):
    print("Tag ",tagset[i])
    print("Precision: ",precision_class[i])
    print("Recall: ",recall_class[i])
    print("f1 score: ",f1_class[i])
    print("\n")


Model metrics

In [ ]:
model_statstics(precision, recall, f1, f0_5, f2, accuracy,precision_class,recall_class,f1_class)

Global statistics


Accuracy:  0.5844367225143063
Precision:  0.5801361729613262
Recall:  0.5844367225143063
F Score
f1 score:  0.560221713725457
f0_5 score:  0.561374688233796
f2_score:  0.5710183947511898


Class wise stats


Tag  .
Precision:  0.5444976076555024
Recall:  0.8903817617698637
f1 score:  0.6757512157665728


Tag  ADJ
Precision:  0.4815889310421781
Recall:  0.35215404699738906
f1 score:  0.4068243943821285


Tag  ADP
Precision:  0.5786843171687154
Recall:  0.546242774566474
f1 score:  0.5619957583575661


Tag  ADV
Precision:  0.3443223443223443
Recall:  0.11273517727306799
f1 score:  0.1698571347902197


Tag  CONJ
Precision:  0.5619694397283531
Recall:  0.04439973172367539
f1 score:  0.08229736449527598


Tag  DET
Precision:  0.6468145099832343
Recall:  0.6637600688198952
f1 score:  0.6551777374657456


Tag  NOUN
Precision:  0.630555982774531
Recall:  0.7513056624518967
f1 score:  0.6856551551133039


Tag  NUM
Precision:  0.3706293706293706
Recall:  0.17785234899328858
f

TypeError: ignored